In [2]:
import pycolmap


In [3]:
from pathlib import Path

In [4]:
!nvidia-smi

Thu Mar 14 18:48:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX150           Off | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              N/A /   9W |      1MiB /  2048MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Prepare Paths

In [5]:
data_path = Path("../data/images")
colmap_path = Path("../data/colmap")

data_path.mkdir(parents=True, exist_ok=True)

# colmap directory should be empty
# colmap_path.mkdir(parents=True, exist_ok=True)
# import shutil
# shutil.rmtree(str(colmap_path))
# colmap_path.mkdir()

In [6]:
output_path=  colmap_path
image_dir =  data_path

mvs_path = output_path / "mvs"
database_path = output_path / "database.db"


# match and extract features

In [6]:
pycolmap.extract_features(database_path, image_dir)
pycolmap.match_exhaustive(database_path)

W20240314 18:13:23.014787   383 feature_extraction.cc:403] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240314 18:13:23.015396   407 misc.cc:198] 
Feature extraction
I20240314 18:13:23.435521   424 feature_extraction.cc:254] Processed file [1/3]
I20240314 18:13:23.435585   424 feature_extraction.cc:257]   Name:            la-statua.jpg
I20240314 18:13:23.435599   424 feature_extraction.cc:283]   Dimensions:      550 x 367
I20240314 18:13:23.435612   424 feature_extraction.cc:286]   Camera:          #3 - SIMPLE_RADIAL
I20240314 18:13:23.435626   424 feature_extraction.cc:289]   Focal Length:    660.00px
I20240314 18:13:23.435643   424 feature_extractio

In [7]:
maps = pycolmap.incremental_mapping(database_path, image_dir, output_path)
maps[0].write(output_path)


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.739102e+01    0.00e+00    7.70e+02   0.00e+00   0.00e+00  1.00e+04        0    1.15e-03    2.92e-03
   1  1.151829e+01    5.87e+00    1.94e+02   4.37e+00   9.61e-01  3.00e+04        1    5.96e-04    3.55e-03
   2  1.124356e+01    2.75e-01    2.06e+02   1.70e+00   1.00e+00  9.00e+04        1    3.81e-04    3.95e-03
   3  1.122581e+01    1.78e-02    9.05e+02   2.75e+00   5.52e-01  9.01e+04        1    5.25e-04    4.48e-03
   4  1.118593e+01    3.99e-02    5.52e+02   2.14e+00   9.71e-01  2.70e+05        1    5.41e-04    5.03e-03
   5  1.121748e+01   -3.15e-02    5.52e+02   4.53e+00  -9.76e-01  1.35e+05        1    1.42e-04    5.19e-03
   6  1.116569e+01    2.02e-02    7.45e+02   2.72e+00   7.27e-01  1.49e+05        1    4.73e-04    5.67e-03
   7  1.113809e+01    2.76e-02    4.97e+02   2.62e+00   9.72e-01  4.47e+05        1    3.62e-04    6.04e-03
   8  1.113645e+01    1.64e-

I20240314 18:13:26.093765   443 misc.cc:198] 
Loading database
I20240314 18:13:26.095031   443 database_cache.cc:54] Loading cameras...
I20240314 18:13:26.095100   443 database_cache.cc:64]  3 in 0.000s
I20240314 18:13:26.095120   443 database_cache.cc:72] Loading matches...
I20240314 18:13:26.095149   443 database_cache.cc:78]  2 in 0.000s
I20240314 18:13:26.095160   443 database_cache.cc:94] Loading images...
I20240314 18:13:26.095515   443 database_cache.cc:143]  3 in 0.000s (connected 3)
I20240314 18:13:26.095539   443 database_cache.cc:154] Building correspondence graph...
I20240314 18:13:26.095602   443 database_cache.cc:190]  in 0.000s (ignored 0)
I20240314 18:13:26.095615   443 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240314 18:13:26.096112   443 misc.cc:198] 
Finding good initial image pair
I20240314 18:13:26.103179   443 incremental_mapper.cc:404] => No good initial image pair found.
I20240314 18:13:26.103231   443 incremental_mapper.cc:310] => Relaxing the initializatio

In [8]:
# dense reconstruction
pycolmap.undistort_images(mvs_path, output_path, image_dir)
pycolmap.patch_match_stereo(mvs_path)  # requires compilation with CUDA
pycolmap.stereo_fusion(mvs_path / "dense.ply", mvs_path)

I20240314 18:13:26.170691   383 images.h:110]  => Reconstruction with 2 images and 90 points
I20240314 18:13:26.170811   446 misc.cc:198] 
Image undistortion
I20240314 18:13:26.171325   446 undistortion.cc:210] Undistorting image [1/2]
I20240314 18:13:26.231324   446 undistortion.cc:210] Undistorting image [2/2]
I20240314 18:13:26.260556   446 undistortion.cc:223] Writing reconstruction...
I20240314 18:13:26.262259   446 undistortion.cc:228] Writing configuration...
I20240314 18:13:26.262399   446 undistortion.cc:232] Writing scripts...
I20240314 18:13:26.262692   446 timer.cc:91] Elapsed time: 0.002 [minutes]


AttributeError: module 'pycolmap' has no attribute 'patch_match_stereo'

In [1]:
import colmap

ModuleNotFoundError: No module named 'colmap'

In [10]:
import pycolmap

# Load the COLMAP database
db = pycolmap.Database(str(database_path))

In [26]:
tvg = db.read_two_view_geometry(1,2)

In [69]:
img = db.read_all_images()[0]

In [87]:
img = db.read_image(3)
img.todict()['points2D']

[]

{'image_id': 1,
 'camera_id': 3,
 'name': 'la-statua.jpg',
 'cam_from_world': {'rotation': {'quat': array([0., 0., 0., 1.])},
  'translation': array([0., 0., 0.])},
 'cam_from_world_prior': {'rotation': {'quat': array([nan, nan, nan, nan])},
  'translation': array([nan, nan, nan])},
 'points2D': [],
 'registered': False,
 'num_points3D': 0,
 'num_observations': 0,
 'num_correspondences': 0}